In [1]:
import json

from collections import Counter
from io import StringIO

import jieba
import numpy as np
import pandas as pd

In [2]:
def evaluation_origin_data(origin_data, output_='origin_data_evaluation_res.xls', describe_=['count', 'mean', 'std', 'min', 'max']):
    """
    数据源与格式评价
    origin_data: pd.DataFrame
    output_     : output url: numeric columns describe data
    describe_  : pd.describe params
    """
    results = {}
    origin_dtpyes = dict(origin_data.get_dtype_counts())
    i = 0
    for _type, _count in origin_dtpyes.items():
        print(f'该数据源有 {_count} 列类型为 {_type} 的数据\n')
        results['数据类型'] = {}
        if _type != 'int64' and _type != 'float64':
            non_numeric_cols = origin_data.select_dtypes(include=[_type]).columns
            print(f'该数据源中 {list(non_numeric_cols)} 列的数据为{_type}类型, 其中以下列的数据为混合类型，详情如下: \n')
            
            for _col in list(non_numeric_cols):
                ct = Counter(origin_data[_col].map(lambda x: str(type(x))))
                if len(ct) != 1:
                    i += 1
                    print('    {} 列共{}种类型的数据,具体为：{}'.format(_col, len(ct), str({_dtype: ct[_dtype] for _dtype in ct})))
    if describe_:
        try:
            check_a.select_dtypes(include=['number']).describe().loc[describe_].to_excel(output_, float_format='%.2f')
        except KeyError as e:
            check_a.select_dtypes(include=['number']).describe().to_excel(output_, float_format='%.2f')
            print(f'\n输入describe_参数有误，已使用默认值写入文件\n')
    print(f'\n数值类型的汇总已经输出到文件origin_data_evaluation_res.xls\n')
    print(f'初步估计，该数据源的数据规整度为：{1 - i /origin_data.columns.size:.3f} \n\n')    



In [3]:
def data_to_db_check(cleaned_data, s_t, in_list=[]):
    """若数据源的数据名在prc中有对应值，进行数据值是否为合法数据的评价"""
    legal_cols = {}
    for col in in_list:
        the_row = s_t[s_t['nameChs'] == col].loc[:, ['sampleTypeId', 'name', 'nameChs', 'nameChsAlts', 'dataType', 
                                                                              'allowedDataRange', 'defaultRefRange']].to_dict(orient='records')
        if the_row:
            the_range = json.loads(the_row[0]['allowedDataRange'])
            the_data_type = the_row[0]['dataType']
            if not isinstance(the_range, dict):
                continue
            if the_data_type in ['float', 'int']:
                for k, v in the_range.items():
                    legal_vals = v.split('~')
                    try:
                        if cleaned_data[col].min() < float(legal_vals[0]) or cleaned_data[col].max() > float(legal_vals[1]):
                            # allowedDataRange可能有多个候选项
                            if not col in legal_cols:
                                print(f'{col} 列的值可能不合法，允许值范围为：{v}, 而源数据值范围为{check_a[col].min()} ~ {check_a[col].max()}')
                        else:
                            # 满足 allowedDataRange的范围
                            legal_cols[col] = f'可能单位为:{k}'
                    except ValueError as e:
                        print(f'\n {col} 列: {e}!\n')
            elif the_data_type == 'text':
                if any(v not in set(cleaned_data[col]) for v in the_range):
                    print(f'{col} 列的值可能不合法，允许值为：{the_range}')
    print(f'以下列的数据均为合法数据:\n{legal_cols}')
    print('---------------------------------------------------------------------------------------')

In [4]:
def evaluation_origin_columns_name(origin_data):
    """
    数据名标准化评价:
        若数据源的某column直接在prc-SampleType中存在，则判定为数据源中的某数据名(列名)在prc中存在；
        否则使用结巴分词并配以自定义词库对数据源的该列(column)进行分词，分词后的任意短语若在prc-SampleType中存在，则判定为相似；
        不然判定为无任何相关性
    最后可依据源数据中无相关性的列来配置分词自定义字典，以达到更好的评估效果。
    """
    print('---------------------------------------------------------------------------------------')
    print('数据名标准化评价: ')
    s_t = pd.read_csv('/Users/har/Desktop/T_SampleType_201811121655.csv')
    s_t_set = set(s_t.to_string(columns=['name', 'nameChs', 'nameChsAlts'], header=False, index=False, na_rep='').split())
    i_, s_, in_list, similar_list, unrelated_list = 0, 0, [], [], []

    for col in origin_data.columns:
            if col in s_t_set:
                i_ +=1
                in_list.append(col)
            elif any(x in ','.join(s_t_set) for x in jieba.cut(col)): 
                s_ +=1
                similar_list.append(col)
            else: unrelated_list.append(col)
    print(f'    源数据中约有{i_ / len(origin_data.columns) * 100: .2f}%的数据名(列名)在prc中存在:')
    print('        ', in_list,'\n')
    print(f'    源数据中约有{s_ / len(origin_data.columns) * 100: .2f}%的数据名(列名)与prc中的数据名(列名)相似:')
    print('        ', similar_list, '\n')
    print(f'    源数据中约有{(1 - (i_ + s_) / len(origin_data.columns)) * 100: .2f}%的数据名(列名)被判别为与prc中的数据名(列名)无任何相关性:')
    print('        ', unrelated_list, '\n')
    if in_list:
        print('---------------------------------------------------------------------------------------')
        print('若数据源的数据名在prc中有对应值，进行数据值评价，结果如下：')
        data_to_db_check(origin_data, s_t, in_list)

In [5]:
check_a = pd.read_excel('/Users/har/Desktop/安徽电信2018数据/a中国电信--检验结果统计2018.10.31.xls')
check_b = pd.read_excel('/Users/har/Desktop/安徽电信2018数据/b中国电信安徽分公司---2018.10.31.xls', sheet_name=1)
check_sum = pd.read_excel('/Users/har/Desktop/安徽电信2018数据/sum中国电信安徽分公司--总检结论统计2018.10.31.xls')
merged  = pd.read_excel('/Users/har/Desktop/merged.xls')

## 中国电信--检验结果统计2018.10.31

In [6]:
evaluation_origin_data(check_a, output_='/Users/har/Desktop/安徽电信2018数据/源数据初步评价/中国电信--检验结果统计.xls')
evaluation_origin_columns_name(check_a)

该数据源有 96 列类型为 float64 的数据

该数据源有 10 列类型为 int64 的数据

该数据源有 11 列类型为 object 的数据

该数据源中 ['体检医院', '单位名称', '姓名', '性别', '蛋白质', '葡萄糖1', '红细胞', '白细胞', '管型', '隐血', '甲状腺过氧化酶抗体'] 列的数据为object类型, 其中以下列的数据为混合类型，详情如下: 

    蛋白质 列共2种类型的数据,具体为：{"<class 'str'>": 189, "<class 'float'>": 3}
    葡萄糖1 列共2种类型的数据,具体为：{"<class 'str'>": 189, "<class 'float'>": 3}
    红细胞 列共2种类型的数据,具体为：{"<class 'str'>": 189, "<class 'float'>": 3}
    白细胞 列共2种类型的数据,具体为：{"<class 'str'>": 189, "<class 'float'>": 3}
    管型 列共2种类型的数据,具体为：{"<class 'str'>": 188, "<class 'float'>": 4}
    隐血 列共2种类型的数据,具体为：{"<class 'str'>": 189, "<class 'float'>": 3}
    甲状腺过氧化酶抗体 列共2种类型的数据,具体为：{"<class 'float'>": 190, "<class 'str'>": 2}


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/zn/5yc2lyxn6r5df135z976_cs80000gn/T/jieba.cache



数值类型的汇总已经输出到文件origin_data_evaluation_res.xls

初步估计，该数据源的数据规整度为：0.940 


---------------------------------------------------------------------------------------
数据名标准化评价: 


Loading model cost 1.080 seconds.
Prefix dict has been built succesfully.


    源数据中约有 35.04%的数据名(列名)在prc中存在:
         ['血红蛋白', '红细胞计数', '白细胞计数', '血小板计数', '尿酸', '尿素', '肌酐', '总蛋白', '白蛋白', '球蛋白', '总胆红素', '间接胆红素', '直接胆红素', '癌胚抗原', '前列腺特异性抗原', '总胆固醇', '甘油三酯', '极低密度脂蛋白胆固醇', '红细胞', '白细胞', '管型', '人绒毛膜促性腺激素', '糖类抗原125', '糖类抗原15-3', '神经元特异性烯醇化酶', '糖类抗原19-9', '铁蛋白', '同型半胱氨酸', '糖类抗原50', '非小细胞肺癌相关抗原', '抗甲状腺球蛋白抗体', '甲状腺素', '促甲状腺激素', '游离甲状腺素', '乳酸脱氢酶', '前白蛋白', '碱性磷酸酶', '总胆汁酸', '类风湿因子', '抗核抗体', '尿微量白蛋白'] 

    源数据中约有 59.83%的数据名(列名)与prc中的数据名(列名)相似:
         ['体检号', '体检医院', '检查日期', '年龄', '白/球比例', '谷丙转氨酶', '谷草转氨酶', '甲胎蛋白', '葡萄糖', '乙肝表面抗原', '蛋白质', '葡萄糖1', '颜色', '镜检', '隐血', '性状', '隐血1', '糖类抗原1251', '糖链抗原242', '糖类抗原CA-242', '孕酮Ⅱ', '甲胎蛋白1', '前列腺特异性抗原1', '癌胚抗原1', '游离前列腺特异性抗原', '糖类抗原15-31', '甲胎蛋白2', '糖类抗原19-91', '癌胚抗原2', 'FPSA/TPSA', '铁蛋白1', '游离前列腺特异性抗原1', '神经元特异性烯醇化酶1', '胃泌素释放肽前体', '前列腺特异性抗原2', 'β2微球蛋白', 'β绒毛膜促性腺激素', '游离三碘甲状原氨酸', '三碘甲状原氨酸', '甲状腺过氧化酶抗体', '白蛋白1', 'γ-谷氨酰转移酶', '总胆红素1', '总蛋白1', '白/球比例1', '直接胆红素1', '间接胆红素1', '谷草转氨酶1', '球蛋白1', '谷丙转氨酶1', '乙肝前S1抗原', '乙肝表面抗原1', '乙肝表面抗体', '乙肝核

## 中国电信安徽分公司---2018.10.31

In [7]:
evaluation_origin_data(check_b, output_='/Users/har/Desktop/安徽电信2018数据/源数据初步评价/中国电信安徽分公司.xls')
evaluation_origin_columns_name(check_b)

该数据源有 40 列类型为 float64 的数据

该数据源有 3 列类型为 int64 的数据

该数据源有 27 列类型为 object 的数据

该数据源中 ['体检医院', '单位名称', '姓名', '性别', '血压', '内科结论', '外科结论', '眼科结论', '耳鼻喉科结论', '口腔科结论', '妇科结论', '妇科TCT', '肺部CT', '肺部CT检查所见', '心电图结论', '双侧乳腺彩超', '双侧乳腺彩超检查所见', '颈动脉彩超', '颈动脉彩超检查所见', '甲状腺彩超', '甲状腺彩超所见', '心脏彩超', '心脏彩超所见', '颈椎MRI', '颈椎MRI所见', '头颅MRI', '头颅MRI所见'] 列的数据为object类型, 其中以下列的数据为混合类型，详情如下: 

    血压 列共2种类型的数据,具体为：{"<class 'str'>": 189, "<class 'float'>": 3}
    妇科结论 列共2种类型的数据,具体为：{"<class 'float'>": 149, "<class 'str'>": 43}
    妇科TCT 列共2种类型的数据,具体为：{"<class 'float'>": 149, "<class 'str'>": 43}
    肺部CT 列共2种类型的数据,具体为：{"<class 'str'>": 186, "<class 'float'>": 6}
    肺部CT检查所见 列共2种类型的数据,具体为：{"<class 'str'>": 186, "<class 'float'>": 6}
    心电图结论 列共2种类型的数据,具体为：{"<class 'str'>": 190, "<class 'float'>": 2}
    双侧乳腺彩超 列共2种类型的数据,具体为：{"<class 'float'>": 191, "<class 'str'>": 1}
    双侧乳腺彩超检查所见 列共2种类型的数据,具体为：{"<class 'float'>": 191, "<class 'str'>": 1}
    颈动脉彩超 列共2种类型的数据,具体为：{"<class 'float'>": 141, "<class 'str'>": 51}
    

## 中国电信安徽分公司--总检结论统计2018.10.31

In [8]:
evaluation_origin_data(check_sum, output_='/Users/har/Desktop/安徽电信2018数据/源数据初步评价/中国电信安徽分公司--总检结论统计.xls')
evaluation_origin_columns_name(check_sum)

该数据源有 1 列类型为 float64 的数据

该数据源有 2 列类型为 int64 的数据

该数据源有 5 列类型为 object 的数据

该数据源中 ['姓名', '部门名称', '性别', '综述', '单位名称'] 列的数据为object类型, 其中以下列的数据为混合类型，详情如下: 

    部门名称 列共2种类型的数据,具体为：{"<class 'float'>": 29, "<class 'str'>": 163}

数值类型的汇总已经输出到文件origin_data_evaluation_res.xls

初步估计，该数据源的数据规整度为：0.875 


---------------------------------------------------------------------------------------
数据名标准化评价: 
    源数据中约有 0.00%的数据名(列名)在prc中存在:
         [] 

    源数据中约有 25.00%的数据名(列名)与prc中的数据名(列名)相似:
         ['体检编号', '年龄'] 

    源数据中约有 75.00%的数据名(列名)被判别为与prc中的数据名(列名)无任何相关性:
         ['姓名', '电话号码', '部门名称', '性别', '综述', '单位名称'] 



## join后的表格

In [9]:
evaluation_origin_data(merged, output_='/Users/har/Desktop/安徽电信2018数据/源数据初步评价/join后的表格.xls')
evaluation_origin_columns_name(merged)

该数据源有 133 列类型为 float64 的数据

该数据源有 12 列类型为 int64 的数据

该数据源有 38 列类型为 object 的数据

该数据源中 ['体检医院', '单位名称', '姓名', '性别', '蛋白质', '葡萄糖1', '红细胞', '白细胞', '管型', '隐血', '甲状腺过氧化酶抗体', '血压', '内科结论', '外科结论', '眼科结论', '耳鼻喉科结论', '口腔科结论', '妇科结论', '妇科TCT', '肺部CT', '肺部CT检查所见', '心电图结论', '双侧乳腺彩超', '双侧乳腺彩超检查所见', '颈动脉彩超', '颈动脉彩超检查所见', '甲状腺彩超', '甲状腺彩超所见', '心脏彩超', '心脏彩超所见', '颈椎MRI', '颈椎MRI所见', '头颅MRI', '头颅MRI所见', 'a_b_merge', '部门名称', '综述', 'ab_sum_merge'] 列的数据为object类型, 其中以下列的数据为混合类型，详情如下: 

    蛋白质 列共2种类型的数据,具体为：{"<class 'str'>": 187, "<class 'float'>": 3}
    葡萄糖1 列共2种类型的数据,具体为：{"<class 'str'>": 187, "<class 'float'>": 3}
    红细胞 列共2种类型的数据,具体为：{"<class 'str'>": 187, "<class 'float'>": 3}
    白细胞 列共2种类型的数据,具体为：{"<class 'str'>": 187, "<class 'float'>": 3}
    管型 列共2种类型的数据,具体为：{"<class 'str'>": 186, "<class 'float'>": 4}
    隐血 列共2种类型的数据,具体为：{"<class 'str'>": 187, "<class 'float'>": 3}
    甲状腺过氧化酶抗体 列共2种类型的数据,具体为：{"<class 'float'>": 188, "<class 'str'>": 2}
    血压 列共2种类型的数据,具体为：{"<class 'str'>": 187, "<class 'float'